# 🖼️ Meme Generator Lab: Student Lab
## Getting Started
Ref repository: https://github.com/IFML-UT/MLLAcademy-2025

**What we're going to do inside of this notebook:**

This notebook is used to simulate and guide your interactions with the meme generator pipeline.
We're going to walk through each step behind generating a meme - why a meme? --> it's **multi-modal!**

- We're going to use **natural language processing (NLP)** to generate new text on a topic of your choosing
- We'll then take this generated caption and find the best image match for the text - this "text to image" type of generative AI is referred to as multi-modal. 
- Lastly, you'll use python libraries within this lab to bring the text and the image together to create a unique meme. 

## 🗺️ Roadmap for the entire lab:
> The lab is broken down into 3 major sections:

1. **Working with LLMs & Inference**: You'll use an open source large language model (LLM) to generate a meme based on general topics or themes. You'll select the best caption from 3 results. We'll be using Meta's `Llama` model family for this text generation, and can experiment with others as well. 


2. **Multi-modal Generative AI:** With your caption from the first part of the lab, you will use OpenCLIP to query the top 3 matches from a library of popular meme images to select the best image, based on your text caption. You'll understand what multi-modal means, and how a pre-trained vision transformer model like `ViT-B-32` can return relevant images based on text inputs.

3. **Combine both the generated text and best image into your final AI-meme for your finished product.** You'll have the option of sharing your favorite meme with the class by uploading it to our shared drive for the week. 


### ⚙️ How It Works:
- Inputs a freeform meme idea or phrase
- Classifies it into a pre-approved topic
- Uses LLaMA 3.1 8B Instruct (via Hugging Face) to generate 3 clean meme captions
- Filters for profanity or off-topic content

### For use later (after this lab) on your own: 
If you would like to clone this repo to your own computer, and run it later you can use the following command to copy it to your machine: 


-- Run this cell first to install dependencies and import necessary modules.
- This `git clone` command will create a folder called MLLAcademy-2025 in your current directory. 
- Note: you'll need to supply your own API tokens for access to any paid models, or to Hugging Face for any open-source hosted model.

> `!git clone https://github.com/IFML-UT/MLLAcademy-2025.git`


## 1️⃣ Load Requirements & Configure Hugging Face Inference API Token: 
To help keep this lab computationally light and flexible for our lab use, we are using Hugging Face inference token (generated by IFML) for your use during this week. 
- API stands for application programing interface, once configured it allows two different software applications communicate and send data to one another. 
- This secret token will expire after this week. 
- If you would like to continue to run this lab later on your own, you can do so by creating a free HuggingFace account, creating a token within the free tier (https://huggingface.co/settings/tokens) and then pasting your new token into the cell's `getpass` feature below. 

Paste your Hugging Face API token (provided to you) in the cell below when prompted. 

> If you don't have one because you are trying this lab outside of our scheduled session no worries! 
> Visit https://huggingface.co/settings/tokens to create a free account, create a token of `type = READ`, and then copy your access token.

In [ ]:
# Clone the repo to this notebook's runtime:
import os
!git clone https://github.com/IFML-UT/MLLAcademy-2025.git

In [ ]:
# This script auto-detects your environment (Colab or local) and configures everything accordingly.
# --- We're going to run this in Google Colab today ---

import sys
import re
import json
from pathlib import Path
from getpass import getpass

# --- Detect Environment ---
def get_runtime_env():
    try:
        import google.colab
        return "colab"
    except ImportError:
        return "local"

env = get_runtime_env()
print(f"Detected environment: {env}")

# --- Install Dependencies ---
if env == "colab":
    %pip install -r /content/MLLAcademy-2025/requirements.txt
else:
    req_path = str(Path("../requirements.txt").resolve())
    %pip install -r $req_path

# --- Hugging Face Token Management ---
token_path = Path("/content/hf_token.txt") if env == "colab" else Path("../hf_token.txt")

if not token_path.exists():
    print("Please enter your Hugging Face API token:")
    token = getpass("Hugging Face Token: ")
    with open(token_path, "w") as f:
        f.write(token.strip())
    print(f"✅ Hugging Face token saved to {token_path}")
else:
    print(f"✅ Hugging Face token found at {token_path}")

# --- Ensure utils folder is in sys.path ---
sys.path.append(str(Path("/content/MLLAcademy-2025/utils").resolve()) if env == "colab" else str(Path("../utils").resolve()))

# --- Import the Safe Caption Generator ---
from safe_caption_generator import safe_caption_generator
print("\nSafe Caption Generator module imported successfully, ready to use!")


Detected environment: local

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
ERROR: Could not open requirements file: [Errno 2] No such file or directory: '/Users/jwhorley/Downloads/requirements.txt'
Note: you may need to restart the kernel to use updated packages.
Please enter your Hugging Face API token:


#### After running the cell above, you should see the following three statements print to the notebook:

> `✅ Hugging Face token found at ../hf_token.txt`<br>
> `Loading embedding model for semantic topic matching...`

> `Safe Caption Generator module imported successfully, ready to use!`

In [ ]:
# Helper function - for printing captions cleanly and export the results to a JSON file
# This file will be used later when we generate the images 
# --- Run this cell ---

def print_captions(captions):
    env = get_runtime_env()
    captions_path = Path("/content/MLLAcademy-2025/captions.json") if env == "colab" else Path("../captions.json")

    with open(captions_path, "w") as f:
        json.dump(captions, f)

    print(f"✅ Captions saved to {captions_path}")
    print("\n---\n\n")
    for i, c in enumerate(captions, 1):
        print(f"Caption {i}: {c}\n")


## 2️⃣ Tests Different Prompts:
This will assign your topic to the variable `user_input`
 - Running the cell below will then run the caption generator and print the captions
 
Additionally, we are going to be using a Python function called `safe_caption_generator` to assist us in prompting the LLM. For example, this code is within the function and prompts the LLM prior to its text generation, based on your input: 

```
PROMPT_TEMPLATE = (
    "Write a short, funny meme caption about this topic: {user_input}.\n"
    "Only return a single caption, in quotes, with no explanation or extra text."
)
```

### We are going to specifically guide our text generation to stay aligned on certain topics.
You may find that certain topics will be blocked from use. If you run into a "try again error message" please adjust your input. Here are the broad topics we are going to use within this lab for your captions: 
- "final exams"
- "group projects"
- "studying late", 
- "Monday mornings"
- "school cafeteria food"
- "summer break"
- "forgetting your homework"
- "getting a pop quiz"
- "trying to stay awake in class"
- "sports"
- "coding projects"
- "hackathons"
- "hanging out with friends"
- "summer weather"
- "family vacations"
- "college applications"
-  "video games"

_You don't have to use these exact words in your `user_input`, but it needs to be semantically similar. For example, "Going to a baseball game instead of studying" would match our themes of both `sports` and `forgetting your homework`, and possibly even `studying late`._

 > Note: This cell may take anywhere from 30 seconds to 2 minutes depending on your prompt and notebook compute resources at the time of execution.

In [ ]:
# --- Now we are going to run the safe caption generator based on your input ---
# In this cell, we'll test our `safe_caption_generator` function with a sample input. It will:
#   - Use your input prompt.
#   - Check if the input matches approved topics.
#   - Generate 3 captions using a language model.
#   - Save the captions to a JSON file for use in later cells.

try:
    # modify this input to test different prompts 
    user_input = "your prompt theme or topic here"
    print(f"Testing prompt: '{user_input}'")
    
    # Generate and save 3 meme captions and print each
    captions = safe_caption_generator(user_input, num_captions=3)
    print_captions(captions)

except ValueError as e:
    print(f"⚠️ Error: {e}")

## 3️⃣ Generate "top 3" captions for your meme
Use the box below to enter your meme idea, click "Generate," and see three captions!

**This specific cell below will save to `captions.json` for use in the next part of the lab.** 

Each new generation overwrites the previous contents of that file. Feel free to use the cell above this one to get a feel for how much (or how little) detail on your topic you want to include in your prompt and observe the quality of the LLM response across the 3 caption options.

If you want to save any specific caption, save it in a new file within your directory. You'll have a chance to select your favorite caption in the next lab. 

In [ ]:
# Interactive Prompt & Demo:
from IPython.display import display
import ipywidgets as widgets

input_box = widgets.Text(value='', placeholder='Enter your meme idea...', description='Prompt:')
run_button = widgets.Button(description="Generate")
output = widgets.Output()

def run_on_click(b):
    output.clear_output()
    with output:
        try:
            captions = safe_caption_generator(input_box.value)
            # for idx, c in enumerate(captions, 1): # backup code to print each caption rather than use function
            #   print(f"{idx}. {c}")
            print_captions(captions)
        except Exception as e:
            print(f"⚠️ Error: {e}")

run_button.on_click(run_on_click)
display(input_box, run_button, output)

## 4️⃣ Troubleshooting Guide

- If you get a profanity or topic error, verify the input is:
  - Clean (no banned phrases)
  - Topically close to: studying, group projects, sports, coding, school, etc.

- If you get an API error:
  - Ensure `hf_token.txt` exists and contains a valid Hugging Face token; if the token is missing, please ask for a new token.
  - Ensure `.gitignore` excludes it from version control

- If you get no captions back:
  - Check output formatting with `print(repr(captions))`
  - Rerun cell — model output may vary by seed

## 5️⃣ Checkpoint Complete! 

✅ LLM text generation complete! You have successfully: 

1. Invoked an open-source LLM via the Hugging Face API and generated text using a cloud-based inference service. 
2. Observed how different prompting can result in different text results. 
3. Generated a set of 3 captions based on a topic that have been saved in your directory as: `captions.json` - find and open that file, and you'll see your three captions. 

## Next Steps:
#### Continue to the image generation portion of the lab below!

# 6️⃣ Meme Image Selector with OpenCLIP 🖼️

In this portion of the notebook, you'll take your meme **captions** from the previous notebook, select one of them, and use it to semantically search a pool of 35 preloaded images.

You'll use OpenCLIP (an open-source model trained to match images to text) to:
- Embed your caption
- Compare it to image embeddings
- Rank the most relevant images
- Visualize the top 3 results

> Learn more about OpenCLIP here: https://github.com/mlfoundations/open_clip
"CLIP" stands for Contrastive Language-Image Pre-training, and OpenCLIP is an open-source alternative
to OpenAI's CLIP model. 

This notebook sets you up to later combine the image + caption into a final meme.

In [ ]:
# Environment Setup
# 🛠️ Install required libraries
# These libraries are required for the code to run. We are going to use "pip" to install them.

!pip install -q \
  open_clip_torch \
  torchvision \
  ftfy \
  regex \
  tqdm \
  matplotlib

# Imports and Model Setup
# Now we are going to import the required libraries and set up the model.
# We are going to use the "open_clip" library to load the CLIP model.

import os
import torch
import open_clip
from PIL import Image
from pathlib import Path
from torchvision import transforms
from tqdm import tqdm

## 7️⃣ Load OpenCLIP: model (ViT-B-32)
This model has been trained on a dataset of **34 billion** image<>caption pairs, and has a 72.8% zero-shot accuracy. 

> http://github.com/mlfoundations/open_clip

In the cell below you'll load and assign the model to variable `model`. 

In [ ]:
# Load OpenCLIP model (ViT-B-32 for speed, pre-trained on LAION-2B)

model, _, preprocess = open_clip.create_model_and_transforms("ViT-B-32", pretrained="laion2b_s34b_b79k")
tokenizer = open_clip.get_tokenizer("ViT-B-32")
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

## 🎱 Getting started with image search: 🔎
Now we are going to load the images that we want to use for the lab.
 - We are going to use the "Path" library to load the images from the directory.

The "glob" method to load all the images from the directory, and we're using the "assert" method to check if the images are loaded correctly - this is an example of error handling within Python. 

In [ ]:
# We are going to use the "Path" library to load the images from the directory.

data_dir = Path("/content/MLLAcademy-2025/images")  # 35 preloaded images for lab
image_paths = list(data_dir.glob("*.jpg")) + list(data_dir.glob("*.png"))
assert len(image_paths) >= 1, "No images found in image directory."

## 9️⃣ Create a function to view the selected images in the notebook

For this task, we'll use a popular Python libray in both machine learning and data science: `matplotlib`. 
You can think of it as a set of tools that allow you to visualize and create graphs and images of data. In this case, our data are the images returned by our `model`. 

We're using  `f string` to format (and later print) the `score` of each image returned. This score represents the model's confidence that image is a good match, based on the input. This is commonly coded as: `f"string {expression}"`

In [ ]:
# --- Run this cell ---
# Helper Function:  Display top matches

import matplotlib.pyplot as plt

def show_images(image_scores, top_k=3):
    top_images = sorted(image_scores, key=lambda x: x[1], reverse=True)[:top_k]
    fig, axes = plt.subplots(1, top_k, figsize=(5 * top_k, 5))
    if top_k == 1:
        axes = [axes]
    for ax, (img_path, score) in zip(axes, top_images):
        ax.imshow(Image.open(img_path))
        ax.set_title(f"Score: {score:.2f}")
        ax.axis("off")
    plt.tight_layout()
    plt.show()

## 🔟 Running an image search based on your inputs: 

**A look under the hood:** 
This is the core of the semantic image search loop.

Here's what happens:
1. Your input `caption` is turned into a vector (embedding) using OpenCLIP’s text encoder.
2. Each image in the dataset is processed and converted to its own image embedding.
3. The code calculates the similarity between the caption and each image using dot-product (cosine similarity under normalization).
4. Each score tells us how well that image matches the meaning of your caption.
5. We save those (image path, score) pairs to rank them later.

Everything is done inside a `with torch.no_grad()` block so it runs efficiently and avoids memory buildup on GPU (if used).

You can change the `caption` below to try new meme ideas and test the image retrieval accuracy as part of our testing.

In [ ]:
# Example Search: Static Caption
# This is an example search that shows the top 3 images based on a static caption. It does not save the results for the next step.

# Run a search based on text caption: 

caption = "Trying to stay awake in class but the professor’s voice is a lullaby"

with torch.no_grad():
    text_tokens = tokenizer([caption]).to(device)
    text_features = model.encode_text(text_tokens)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    image_scores = []
    for img_path in tqdm(image_paths):
        image = preprocess(Image.open(img_path).convert("RGB")).unsqueeze(0).to(device)
        image_features = model.encode_image(image)
        image_features /= image_features.norm(dim=-1, keepdim=True)

        similarity = (text_features @ image_features.T).item()
        image_scores.append((img_path, similarity))

# Show the top matches using the show_images function from above:
# We're going to return the top 3 matches based on the similarity score.
show_images(image_scores, top_k=3)

## 1️⃣1️⃣ Run an image earch based on the caption you generated earlier: 
We are going to reference the `captions.json` created in the last notebook within this lab. 

This version lets you reuse captions generated earlier by loading them from a saved JSON file.
Make sure `captions.json` exists (created from the previous `1_meme_generator_inst.ipynb` notebook.

> If your captions.json is empty or showing captions different from your last run, go back to the first notebook, restart your kernel, and re run the cells to refresh the data. 

In [ ]:
import json
import ipywidgets as widgets
from IPython.display import display, clear_output

# Load captions.json generated in earlier notebook
captions_file = Path("/content/MLLAcademy-2025/captions.json")
assert captions_file.exists(), "captions.json not found. Run the Instructor Notebook to generate it first."

with open(captions_file, "r") as f:
    caption_options = json.load(f)

caption_dropdown = widgets.Dropdown(
    options=caption_options,
    description='Caption:',
    layout=widgets.Layout(width='100%')
)

run_button = widgets.Button(description="Search Images")
output = widgets.Output()

def on_click(b):
    output.clear_output()
    with output:
        caption = caption_dropdown.value
        print(f"🔎 Searching for: {caption}")

        # Save the selected caption to ../selected_caption.json for use in Notebook B
        with open("/content/MLLAcademy-2025/selected_caption.json", "w") as f:
            json.dump(caption, f)
        print(f"✅ Selected caption saved to '../selected_caption.json': {caption}") # We'll use this .json file in the next notebook

        with torch.no_grad():
            text_tokens = tokenizer([caption]).to(device)
            text_features = model.encode_text(text_tokens)
            text_features /= text_features.norm(dim=-1, keepdim=True)

            image_scores = []
            for img_path in tqdm(image_paths):
                image = preprocess(Image.open(img_path).convert("RGB")).unsqueeze(0).to(device)
                image_features = model.encode_image(image)
                image_features /= image_features.norm(dim=-1, keepdim=True)

                similarity = (text_features @ image_features.T).item()
                image_scores.append((img_path, similarity))

        show_images(image_scores, top_k=3)

        # Sort and save top 3 images for next cell
        image_scores_sorted = sorted(image_scores, key=lambda x: x[1], reverse=True)
        top_image_paths = [str(path) for path, _ in image_scores_sorted[:3]]

        with open("/content/MLLAcademy-2025/top_images.json", "w") as f:
            json.dump(top_image_paths, f)

        print("✅ Top 3 image paths saved to '/content/MLLAcademy-2025/top_images.json' for selection in the next cell.")

run_button.on_click(on_click)
display(caption_dropdown, run_button, output)

## 1️⃣2️⃣ Select the image for your meme from results

The cell below lets you select the final image from the top 3 matches and prepares it for use in the final meme.
You'll choose one of the top image paths to pair with your caption selected from earlier.

Run the cell below to select your image.

**How to Use This Cell**:
- Below, you'll see the 3 best image matches based on your selected meme text.
- Each image has a **'Select This Image'** button beneath it.
- Click the button under the image you like the most.


Your selection will be saved to `/content/MLLAcademy-2025/selected_image.json` for the next part of the lab.

In [ ]:
# This cell lets you select the final image from the top 3 matches by clicking on the image.
# --- Your selection will be saved for use in the final meme. --- 

from IPython.display import display
import ipywidgets as widgets
import json
from PIL import Image
from io import BytesIO
import base64

# Load top 3 image paths from generated JSON (instead of static list)
with open("/content/MLLAcademy-2025/top_images.json", "r") as f:
    top_image_paths = json.load(f)

# Prepare image widgets
def image_to_widget(img_path):
    img = Image.open(img_path).convert("RGB")
    buffer = BytesIO()
    img.thumbnail((400, 400))
    img.save(buffer, format="JPEG")
    encoded = base64.b64encode(buffer.getvalue()).decode("utf-8")
    return widgets.HTML(f'<img src="data:image/jpeg;base64,{encoded}" style="border:2px solid black; margin:5px;">')

# Display images with buttons
output = widgets.Output()

selected_image = None

def on_button_click(img_path):
    global selected_image
    selected_image = img_path
    with output:
        output.clear_output()
        print(f"✅ You selected: {img_path}")
        with open("/content/MLLAcademy-2025/selected_image.json", "w") as f:
            json.dump(selected_image, f)
        print("Saved selection to '/content/MLLAcademy-2025/selected_image.json'.")

for img_path in top_image_paths:
    img_widget = image_to_widget(img_path)
    button = widgets.Button(description="Select This Image", layout=widgets.Layout(width='auto'))
    button.on_click(lambda b, p=img_path: on_button_click(p))
    display(widgets.VBox([img_widget, button]))

display(output)

## What Happens Next: 📝

Once you've selected an image, it will be saved for you.

You'll combine it with your chosen caption in the final lab step to create your meme!

## Checkpoint Complete! Image Ranking with OpenCLIP ✅

You’ve now tested text-to-image semantic matching using OpenCLIP! Congratulations! 

You've completed
- Entering a meme caption, loaded from your `captions.json` file.
- Performed a vector embedding and search of your caption text using OpenCLIP to return the top 3 images based on that caption.
- Displayed the scoring (confidence) of each image.
- Picked an image you want to pair with the caption - these are the building blocks of your AI-meme.

#### Next Steps:
**→ You are now ready to integrate this into the a final product in the next notebook!**

# 1️⃣3️⃣ Final Meme Assembly! 🏗️

Welcome to the final part of your Meme Generator lab! 🎉

In this notebook, you'll combine your selected image with your meme text to create the final meme image. We'll bring together the text caption you generated using an LLM, and the image you selected after searching OpenCLIP for the most suitable match for that caption.

Let's get started by importing the necessary libraries we'll use in this notebook to bring it all together.

In [2]:
# --- Run this cell ---

import json
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display
from pathlib import Path
import textwrap
import ipywidgets as widgets

%pip install ipywidgets


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## 1️⃣4️⃣ Load your selected caption and image
This next cell loads the selections you made in the previous parts of the lab:
- Your chosen meme text from `captions.json`
- The image path you selected from `selected_image.json`
- You *cannot* change your caption here—this ensures consistency with what you searched for earlier!

After you run this cell, you'll be presented with the "selected caption" and the "final meme image".

In [ ]:
# Load caption from the "Selected Caption" file we created earlier
from pathlib import Path

selected_caption_path = Path("/content/MLLAcademy-2025/selected_caption.json")
if not selected_caption_path.exists():
    raise FileNotFoundError(f"{selected_caption_path} not found. Please run the previous cell to generate it.")

with open(selected_caption_path, "r") as f:
    caption = json.load(f)

# Get the first (and only) caption from the file:
selected_caption = caption

print(f"✅ Your selected meme caption: {selected_caption}\n")

# Load selected image
with open("/content/MLLAcademy-2025/selected_image.json", "r") as f:
    selected_image_path = json.load(f)

print(f"✅ Selected image loaded from: {selected_image_path}")

# Display selected image
from PIL import Image
img = Image.open(selected_image_path)
display(img)

FileNotFoundError: /content/MLLAcademy-2025/selected_caption.json not found. Please run the previous cell to generate it.

## 1️⃣5️⃣ Generate the final "Meme Product"

This next cell overlays the selected caption "meme text" onto your chosen image. 
- You can customize the font size and position below w/in this cell using the various code snippets; only uncomment 1 at a time to move the text around. You can do this by adding or removing the `#`'s.

**In this lab:**
- We've added a dynamic font sizing system that scales text based on the image size.
- If **Impact.ttf** is missing, we fall back to the default font.
- You can change the font file and size in the `create_meme()` function!

#### Example: How is the text centered on the image?

We calculate the leftover space on each axis by subtracting the text size from the image size, then divide by 2. 

For example, the code:

    `position = ((image.width - text_width) // 2, (image.height - text_height) // 2)`

finds the horizontal (x) and vertical (y) center points. This is a common approach in pixel-based graphics.

Once you've adjusted the cell's code - run the cell, and then click `Generate` within the widget to display your meme. This will also save a copy of your meme to `my_final_meme.png`. 

#### Tips: 
- The variable `font_size` controls how big your text font is relative to the size of the image. Decreasing this value will decrease the size of your font; increasing this value will increase the size of your font. 

- Feel free to re-run this cell as many times as you need to in order to get the text exactly where you want it. 

- You *cannot* change your caption here—this ensures consistency with what you searched for earlier!

- There is a function called `draw.multiline_text` in the cell below - this is used _both_ for the white text _and_ a black outline that helps contrast the text on the image and ensures it looks consistent with an expected "meme style". It's in this function you can adjust the "left", "right", or "center" justification of the text - or even the color of your text if you wish. It's setup with white text by default.

In [ ]:
# Function to create meme (customizable)
def create_meme(image_path, text, output_path="/content/MLLAcademy-2025/my_final_meme.png", font_size_percent=0.08, position_choice="center", align_choice="center"):
    image = Image.open(image_path).convert("RGB")
    draw = ImageDraw.Draw(image)

    font_path = "/content/MLLAcademy-2025/fonts/impact.ttf"
    font_size = int(image.height * font_size_percent)
    try:
        font = ImageFont.truetype(font_path, font_size)
    except OSError:
        print("⚠️ Impact font not found. Using DejaVuSans-Bold as fallback.")
        from matplotlib import font_manager
        fallback_path = font_manager.findfont("DejaVuSans-Bold")
        font = ImageFont.truetype(fallback_path, font_size)

    # Wrap text
    max_chars_per_line = int(image.width / (font_size * 0.6))
    wrapped_text = textwrap.fill(text, width=max_chars_per_line)
    # text_width, text_height = draw.multiline_textsize(wrapped_text, font=font)
    bbox = draw.multiline_textbbox((0, 0), wrapped_text, font=font)
    text_width = bbox[2] - bbox[0]
    text_height = bbox[3] - bbox[1]

    # Set position options
    positions = {
        "center": ((image.width - text_width) // 2, (image.height - text_height) // 2),
        "top": ((image.width - text_width) // 2, 20),
        "bottom": ((image.width - text_width) // 2, image.height - text_height - 20),
        "top-left": (20, 20),
        "top-right": (image.width - text_width - 20, 20),
        "bottom-left": (20, image.height - text_height - 20),
        "bottom-right": (image.width - text_width - 20, image.height - text_height - 20),
    }
    position = positions.get(position_choice, positions["center"])

    # Draw outline
    outline_color = "black"
    for x in [-2, 0, 2]:
        for y in [-2, 0, 2]:
            draw.multiline_text((position[0] + x, position[1] + y), wrapped_text, font=font, fill=outline_color, align=align_choice)

    # Main text
    draw.multiline_text(position, wrapped_text, font=font, fill="white", align=align_choice)

    image.save(output_path)
    print(f"✅ Meme saved to {output_path}")
    display(image)

# User controls
font_size_slider = widgets.FloatSlider(
    value=0.08,
    min=0.02,
    max=0.15,
    step=0.01,
    description="Font Size %",
    readout_format='.2f',
    layout=widgets.Layout(width='50%')
)

position_choice = widgets.Dropdown(
    options=["center", "top", "bottom", "top-left", "top-right", "bottom-left", "bottom-right"],
    value="center",
    description="Position:",
    layout=widgets.Layout(width='50%')
)

align_choice = widgets.Dropdown(
    options=["center", "left", "right"],
    value="center",
    description="Align:",
    layout=widgets.Layout(width='50%')
)

generate_button = widgets.Button(description="** Create My Meme! **", button_style='success', layout=widgets.Layout(width='50%'))
output = widgets.Output()


def on_generate_click(b):
    output.clear_output()
    with output:
        create_meme(selected_image_path, 
                    selected_caption,
                    font_size_percent=font_size_slider.value,
                    position_choice=position_choice.value, 
                    align_choice=align_choice.value)


generate_button.on_click(on_generate_click)
display(font_size_slider, position_choice, align_choice, generate_button, output)

NameError: name 'widgets' is not defined

# Congratulations! 🎉
### You've completed the Meme Generator Lab!

- Your meme is saved as 'my-final_meme.png`. Feel free to share it, and if you'd like to submit it to our team for display later in the week by uploading it to the form linked on the display in class. 

#### Upload your final meme image here: 
> https://forms.gle/EJgnisAUdNwDZZFH6

If you have time, feel free to run the notebooks again to create more memes if you would like. 

In order to do this, you'll need to start with the begining to first generate a meme caption from your text topic inputs. 

This will overwrite your previous `captions.json`, `selected_caption.json`, `top_images.json` and `selected_image.json` files within this environment. 